# Опять переустонавливать Conda

In [61]:
!nvidia-smi

Mon May 30 07:59:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   46C    P8    19W / 175W |   7047MiB /  8192MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Обучаем модель

In [1]:
import torch
import svox2

In [2]:
%cd svox2/opt
from util.dataset import datasets
from util.util import get_expon_lr_func, generate_dirs_equirect, viridis_cmap

/home/artem/Desktop/coursework/svox2/opt


In [3]:
%cd /home/artem/Desktop/coursework/svox2
PATH = "/home/artem/Desktop/coursework"
device = "cuda" if torch.cuda.is_available() else "cpu"
DATA_PATH = PATH + "/lego_real_night_radial"

/home/artem/Desktop/coursework/svox2


In [64]:
factor = 1
dset = datasets["auto"](
               DATA_PATH,
               split="train",
               device=device,
               factor=factor,
               n_images = 80)

Defaulting to extended NSVF dataset
LOAD NSVF DATA /home/artem/Desktop/coursework/lego_real_night_radial split train


100%|███████████████████████████████████████████| 95/95 [00:02<00:00, 33.35it/s]


NORMALIZE BY? camera
scene_scale 0.5137696314019394
 intrinsics (loaded reso) Intrin(fx=745.3564686190166, fy=745.3564686190166, cx=510.0, cy=384.0)
 Generating rays, scaling factor 1


In [66]:
dset_test = datasets["auto"](DATA_PATH, split="test")

Defaulting to extended NSVF dataset
LOAD NSVF DATA /home/artem/Desktop/coursework/lego_real_night_radial split test


100%|█████████████████████████████████████████████| 7/7 [00:00<00:00, 34.09it/s]

NORMALIZE BY? camera
scene_scale 0.5137696314019394
 intrinsics (loaded reso) Intrin(fx=745.3564686190166, fy=745.3564686190166, cx=510.0, cy=384.0)


In [67]:
reso_list = [[256, 256, 256], [512, 512, 512]]
reso_id = 0

In [68]:
grid = svox2.SparseGrid(reso=reso_list[reso_id],
                        center=dset.scene_center,
                        radius=dset.scene_radius,
                        use_sphere_bound=dset.use_sphere_bound, 
                        basis_dim=9,
                        use_z_order=True,
                        device=device,
                        basis_reso=32,
                        basis_type=svox2.__dict__['BASIS_TYPE_' + "SH"], #['sh', '3d_texture', 'mlp']
                        mlp_posenc_size=4,
                        mlp_width=32,
                        background_nlayers=0,
                        background_reso=512)

In [69]:
grid.sh_data.data[:] = 0.0
grid.density_data.data[:] = 0.0

In [70]:
grid.basis_type == svox2.BASIS_TYPE_3D_TEXTURE

False

In [71]:
grid.requires_grad_(True)
print('Render options', grid.opt)

gstep_id_base = 0
resample_cameras = [
        svox2.Camera(c2w.to(device=device),
                     dset.intrins.get('fx', i),
                     dset.intrins.get('fy', i),
                     dset.intrins.get('cx', i),
                     dset.intrins.get('cy', i),
                     width=dset.get_image_size(i)[1],
                     height=dset.get_image_size(i)[0],
                     ndc_coeffs=dset.ndc_coeffs) for i, c2w in enumerate(dset.c2w)
    ]

Render options RenderOptions(backend='cuvol', background_brightness=1.0, step_size=0.5, sigma_thresh=1e-10, stop_thresh=1e-07, last_sample_opaque=False, near_clip=0.0, use_spheric_clip=False, random_sigma_std=1.0, random_sigma_std_background=1.0)


In [72]:
from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter

In [73]:
summary_writer = SummaryWriter(PATH + "/ckpt/")

In [74]:
import cv2
import math
import gc
import torch.nn.functional as F
import os
from os import path

In [75]:
def train_step():
        print('Train step', torch.cuda.memory_reserved(0), torch.cuda.memory_allocated(0))
        pbar = tqdm(enumerate(range(0, epoch_size, args.batch_size)), total=batches_per_epoch)
        stats = {"mse" : 0.0, "psnr" : 0.0, "invsqr_mse" : 0.0}
        for iter_id, batch_begin in pbar:
            gstep_id = iter_id + gstep_id_base
            if args.lr_fg_begin_step > 0 and gstep_id == args.lr_fg_begin_step:
                grid.density_data.data[:] = args.init_sigma
            lr_sigma = lr_sigma_func(gstep_id) * lr_sigma_factor
            lr_sh = lr_sh_func(gstep_id) * lr_sh_factor
            lr_basis = lr_basis_func(gstep_id - args.lr_basis_begin_step) * lr_basis_factor
            lr_sigma_bg = lr_sigma_bg_func(gstep_id - args.lr_basis_begin_step) * lr_basis_factor
            lr_color_bg = lr_color_bg_func(gstep_id - args.lr_basis_begin_step) * lr_basis_factor
            if not args.lr_decay:
                lr_sigma = args.lr_sigma * lr_sigma_factor
                lr_sh = args.lr_sh * lr_sh_factor
                lr_basis = args.lr_basis * lr_basis_factor

            batch_end = min(batch_begin + args.batch_size, epoch_size)
            batch_origins = dset.rays.origins[batch_begin: batch_end]
            batch_dirs = dset.rays.dirs[batch_begin: batch_end]
            rgb_gt = dset.rays.gt[batch_begin: batch_end]
            rays = svox2.Rays(batch_origins, batch_dirs)

            #  with Timing("volrend_fused"):
            rgb_pred = grid.volume_render_fused(rays, rgb_gt,
                    beta_loss=args.lambda_beta,
                    sparsity_loss=args.lambda_sparsity,
                    randomize=args.enable_random)
            

            #  with Timing("loss_comp"):
            mse = F.mse_loss(rgb_gt, rgb_pred)

            # Stats
            mse_num : float = mse.detach().item()
            psnr = -10.0 * math.log10(mse_num)
            stats['mse'] += mse_num
            stats['psnr'] += psnr
            stats['invsqr_mse'] += 1.0 / mse_num ** 2

            if (iter_id + 1) % args.print_every == 0:
                # Print averaged stats
                pbar.set_description(f'epoch {epoch_id} psnr={psnr:.2f}')
                for stat_name in stats:
                    stat_val = stats[stat_name] / args.print_every
                    summary_writer.add_scalar(stat_name, stat_val, global_step=gstep_id)
                    stats[stat_name] = 0.0
                #  if args.lambda_tv > 0.0:
                #      with torch.no_grad():
                #          tv = grid.tv(logalpha=args.tv_logalpha, ndc_coeffs=dset.ndc_coeffs)
                #      summary_writer.add_scalar("loss_tv", tv, global_step=gstep_id)
                #  if args.lambda_tv_sh > 0.0:
                #      with torch.no_grad():
                #          tv_sh = grid.tv_color()
                #      summary_writer.add_scalar("loss_tv_sh", tv_sh, global_step=gstep_id)
                #  with torch.no_grad():
                #      tv_basis = grid.tv_basis() #  summary_writer.add_scalar("loss_tv_basis", tv_basis, global_step=gstep_id)
                summary_writer.add_scalar("lr_sh", lr_sh, global_step=gstep_id)
                summary_writer.add_scalar("lr_sigma", lr_sigma, global_step=gstep_id)
                if grid.basis_type == svox2.BASIS_TYPE_3D_TEXTURE:
                    summary_writer.add_scalar("lr_basis", lr_basis, global_step=gstep_id)
                if grid.use_background:
                    summary_writer.add_scalar("lr_sigma_bg", lr_sigma_bg, global_step=gstep_id)
                    summary_writer.add_scalar("lr_color_bg", lr_color_bg, global_step=gstep_id)

                if args.weight_decay_sh < 1.0:
                    grid.sh_data.data *= args.weight_decay_sigma
                if args.weight_decay_sigma < 1.0:
                    grid.density_data.data *= args.weight_decay_sh

            #  # For outputting the % sparsity of the gradient
            #  indexer = grid.sparse_sh_grad_indexer
            #  if indexer is not None:
            #      if indexer.dtype == torch.bool:
            #          nz = torch.count_nonzero(indexer)
            #      else:
            #          nz = indexer.size()
            #      with open(os.path.join(args.train_dir, 'grad_sparsity.txt'), 'a') as sparsity_file:
            #          sparsity_file.write(f"{gstep_id} {nz}\n")

            # Apply TV/Sparsity regularizers
            if args.lambda_tv > 0.0:
                #  with Timing("tv_inpl"):
                grid.inplace_tv_grad(grid.density_data.grad,
                        scaling=args.lambda_tv,
                        sparse_frac=args.tv_sparsity,
                        logalpha=args.tv_logalpha,
                        ndc_coeffs=dset.ndc_coeffs,
                        contiguous=args.tv_contiguous)
            if args.lambda_tv_sh > 0.0:
                #  with Timing("tv_color_inpl"):
                grid.inplace_tv_color_grad(grid.sh_data.grad,
                        scaling=args.lambda_tv_sh,
                        sparse_frac=args.tv_sh_sparsity,
                        ndc_coeffs=dset.ndc_coeffs,
                        contiguous=args.tv_contiguous)
            if args.lambda_tv_lumisphere > 0.0:
                grid.inplace_tv_lumisphere_grad(grid.sh_data.grad,
                        scaling=args.lambda_tv_lumisphere,
                        dir_factor=args.tv_lumisphere_dir_factor,
                        sparse_frac=args.tv_lumisphere_sparsity,
                        ndc_coeffs=dset.ndc_coeffs)
            if args.lambda_l2_sh > 0.0:
                grid.inplace_l2_color_grad(grid.sh_data.grad,
                        scaling=args.lambda_l2_sh)
            if grid.use_background and (args.lambda_tv_background_sigma > 0.0 or args.lambda_tv_background_color > 0.0):
                grid.inplace_tv_background_grad(grid.background_data.grad,
                        scaling=args.lambda_tv_background_color,
                        scaling_density=args.lambda_tv_background_sigma,
                        sparse_frac=args.tv_background_sparsity,
                        contiguous=args.tv_contiguous)
            if args.lambda_tv_basis > 0.0:
                tv_basis = grid.tv_basis()
                loss_tv_basis = tv_basis * args.lambda_tv_basis
                loss_tv_basis.backward()
            #  print('nz density', torch.count_nonzero(grid.sparse_grad_indexer).item(),
            #        ' sh', torch.count_nonzero(grid.sparse_sh_grad_indexer).item())

            # Manual SGD/rmsprop step
            if gstep_id >= args.lr_fg_begin_step:
                grid.optim_density_step(lr_sigma, beta=args.rms_beta, optim=args.sigma_optim)
                grid.optim_sh_step(lr_sh, beta=args.rms_beta, optim=args.sh_optim)
            if grid.use_background:
                grid.optim_background_step(lr_sigma_bg, lr_color_bg, beta=args.rms_beta, optim=args.bg_optim)
            if gstep_id >= args.lr_basis_begin_step:
                if grid.basis_type == svox2.BASIS_TYPE_3D_TEXTURE:
                    grid.optim_basis_step(lr_basis, beta=args.rms_beta, optim=args.basis_optim)
                elif grid.basis_type == svox2.BASIS_TYPE_MLP:
                    optim_basis_mlp.step()
                    optim_basis_mlp.zero_grad()

In [76]:
# Test
def eval_step():
    # Put in a function to avoid memory leak
    print('Eval step')
    with torch.no_grad():
        stats_test = {'psnr' : 0.0, 'mse' : 0.0}

        # Standard set
        N_IMGS_TO_EVAL = min(20 if epoch_id > 0 else 5, dset_test.n_images)
        N_IMGS_TO_SAVE = N_IMGS_TO_EVAL # if not args.tune_mode else 1
        img_eval_interval = dset_test.n_images // N_IMGS_TO_EVAL
        img_save_interval = (N_IMGS_TO_EVAL // N_IMGS_TO_SAVE)
        img_ids = range(0, dset_test.n_images, img_eval_interval)

        # Special 'very hard' specular + fuzz set
        #  img_ids = [2, 5, 7, 9, 21,
        #             44, 45, 47, 49, 56,
        #             80, 88, 99, 115, 120,
        #             154]
        #  img_save_interval = 1

        n_images_gen = 0
        for i, img_id in tqdm(enumerate(img_ids), total=len(img_ids)):
            c2w = dset_test.c2w[img_id].to(device=device)
            cam = svox2.Camera(c2w,
                                dset_test.intrins.get('fx', img_id),
                                dset_test.intrins.get('fy', img_id),
                                dset_test.intrins.get('cx', img_id),
                                dset_test.intrins.get('cy', img_id),
                                width=dset_test.get_image_size(img_id)[1],
                                height=dset_test.get_image_size(img_id)[0],
                                ndc_coeffs=dset_test.ndc_coeffs)
            rgb_pred_test = grid.volume_render_image(cam, use_kernel=True)
            rgb_gt_test = dset_test.gt[img_id].to(device=device)
            all_mses = ((rgb_gt_test - rgb_pred_test) ** 2).cpu()
            if i % img_save_interval == 0:
                img_pred = rgb_pred_test.cpu()
                img_pred.clamp_max_(1.0)
                summary_writer.add_image(f'test/image_{img_id:04d}',
                        img_pred, global_step=gstep_id_base, dataformats='HWC')
                if args.log_mse_image:
                    mse_img = all_mses / all_mses.max()
                    summary_writer.add_image(f'test/mse_map_{img_id:04d}',
                            mse_img, global_step=gstep_id_base, dataformats='HWC')
                if args.log_depth_map:
                    depth_img = grid.volume_render_depth_image(cam,
                                args.log_depth_map_use_thresh if
                                args.log_depth_map_use_thresh else None
                            )
                    depth_img = viridis_cmap(depth_img.cpu())
                    summary_writer.add_image(f'test/depth_map_{img_id:04d}',
                            depth_img,
                            global_step=gstep_id_base, dataformats='HWC')

            rgb_pred_test = rgb_gt_test = None
            mse_num : float = all_mses.mean().item()
            psnr = -10.0 * math.log10(mse_num)
            if math.isnan(psnr):
                print('NAN PSNR', i, img_id, mse_num)
                assert False
            stats_test['mse'] += mse_num
            stats_test['psnr'] += psnr
            n_images_gen += 1

        if grid.basis_type == svox2.BASIS_TYPE_3D_TEXTURE or \
            grid.basis_type == svox2.BASIS_TYPE_MLP:
              # Add spherical map visualization
            EQ_RESO = 256
            eq_dirs = generate_dirs_equirect(EQ_RESO * 2, EQ_RESO)
            eq_dirs = torch.from_numpy(eq_dirs).to(device=device).view(-1, 3)

            if grid.basis_type == svox2.BASIS_TYPE_MLP:
                sphfuncs = grid._eval_basis_mlp(eq_dirs)
            else:
                sphfuncs = grid._eval_learned_bases(eq_dirs)
            sphfuncs = sphfuncs.view(EQ_RESO, EQ_RESO*2, -1).permute([2, 0, 1]).cpu().numpy()

            stats = [(sphfunc.min(), sphfunc.mean(), sphfunc.max())
                    for sphfunc in sphfuncs]
            sphfuncs_cmapped = [viridis_cmap(sphfunc) for sphfunc in sphfuncs]
            for im, (minv, meanv, maxv) in zip(sphfuncs_cmapped, stats):
                cv2.putText(im, f"{'%.4f' % minv} {'%.4f' % meanv} {'%.4f' % maxv}", (10, 20),
                            0, 0.5, [255, 0, 0])
            sphfuncs_cmapped = np.concatenate(sphfuncs_cmapped, axis=0)
            summary_writer.add_image(f'test/spheric',
                    sphfuncs_cmapped, global_step=gstep_id_base, dataformats='HWC')
            # END add spherical map visualization

        stats_test['mse'] /= n_images_gen
        stats_test['psnr'] /= n_images_gen
        for stat_name in stats_test:
            summary_writer.add_scalar('test/' + stat_name,
                    stats_test[stat_name], global_step=gstep_id_base)
        summary_writer.add_scalar('epoch_id', float(epoch_id), global_step=gstep_id_base)
        print('eval stats:', stats_test)

In [77]:
class HandMadeArgs:
    lr_decay = True

    sigma_optim = ['sgd', 'rmsprop'][1]
    lr_fg_begin_step = 0 
    lr_sigma = 3e1
    lr_sigma_final =  5e-2
    lr_sigma_delay_steps =  250000
    lr_sigma_delay_mult =  1e-2
    lr_sigma_decay_steps =  15000

    sh_optim = ['sgd', 'rmsprop'][1]
    lr_sh = 1e-2
    lr_sh_final = 5e-6
    lr_sh_decay_steps = 250000
    lr_sh_delay_mult = 1e-2
    lr_sh_delay_steps= 0

    basis_optim = ['sgd', 'rmsprop'][1]
    lr_basis_begin_step = 0
    lr_basis = 1e-6
    lr_basis_final = 1e-6
    lr_basis_decay_steps = 250000
    lr_basis_delay_mult = 1e-2
    lr_basis_delay_steps = 0

    lr_color_bg = 1e-1
    lr_color_bg_final = 5e-6
    lr_color_bg_decay_steps = 250000
    lr_color_bg_delay_mult = 1e-2
    lr_color_bg_delay_steps = 0

    lr_sigma_bg = 3e0
    lr_sigma_bg_final = 3e-3
    lr_sigma_bg_decay_steps = 250000
    lr_sigma_bg_delay_mult = 1e-2
    lr_sigma_bg_delay_steps = 0

    batch_size = 5000

    log_depth_map_use_thresh = None

    rms_beta = 0.95

    print_every = 20
    save_every = 5
    eval_every = 1

    upsamp_every =  3 * 12800

    lambda_beta = 0.0
    lambda_sparsity = 0.0
    enable_random = True
    
    log_mse_image = False
    log_depth_map = False
    log_depth_map_use_thresh = None


    lambda_tv = 1e-5
    tv_sparsity = 0.01
    tv_logalpha = False
    lambda_tv_sh = 1e-3
    tv_sh_sparsity = 0.01
    lambda_tv_lumisphere = 0.0
    tv_lumisphere_sparsity = 0.01
    tv_lumisphere_dir_factor = 0.0
    tv_decay = 1.0
    lambda_l2_sh = 0.0
    tv_early_only = 1
    tv_contiguous = 1

    lambda_tv_basis = 0.0

    init_sigma = 0.1
    init_sigma_bg = 0.1

    thresh_type = ["weight", "sigma"][0]
    weight_thresh = 0.0005 * 512
    density_thresh = 5
    background_density_thresh = 1.0+1e-9
    max_grid_elements = 44_000_000 # 22G

    tune_mode = False
    tune_nosave = False

    weight_decay_sh = 1
    weight_decay_sigma = 1

    n_iters = 10 * 12800
    upsample_density_add = 0.0

    crop = 1.0
    ray_len = False
    no_vid = False
    no_imsave = False

args = HandMadeArgs()

In [78]:
ckpt_path = PATH + "/ckpt.npz"

lr_sigma_func = get_expon_lr_func(args.lr_sigma, args.lr_sigma_final, args.lr_sigma_delay_steps,
                                  args.lr_sigma_delay_mult, args.lr_sigma_decay_steps)
lr_sh_func = get_expon_lr_func(args.lr_sh, args.lr_sh_final, args.lr_sh_delay_steps,
                               args.lr_sh_delay_mult, args.lr_sh_decay_steps)
lr_basis_func = get_expon_lr_func(args.lr_basis, args.lr_basis_final, args.lr_basis_delay_steps,
                               args.lr_basis_delay_mult, args.lr_basis_decay_steps)
lr_sigma_bg_func = get_expon_lr_func(args.lr_sigma_bg, args.lr_sigma_bg_final, args.lr_sigma_bg_delay_steps,
                               args.lr_sigma_bg_delay_mult, args.lr_sigma_bg_decay_steps)
lr_color_bg_func = get_expon_lr_func(args.lr_color_bg, args.lr_color_bg_final, args.lr_color_bg_delay_steps,
                               args.lr_color_bg_delay_mult, args.lr_color_bg_decay_steps)
lr_sigma_factor = 1.0
lr_sh_factor = 1.0
lr_basis_factor = 1.0

last_upsamp_step = 0

epoch_id = -1

In [79]:
if grid.basis_type == svox2.BASIS_TYPE_3D_TEXTURE:
    grid.reinit_learned_bases(init_type='sh')
    #  grid.reinit_learned_bases(init_type='fourier')
    #  grid.reinit_learned_bases(init_type='sg', upper_hemi=True)
    #  grid.basis_data.data.normal_(mean=0.28209479177387814, std=0.001)

elif grid.basis_type == svox2.BASIS_TYPE_MLP:
    # MLP!
    optim_basis_mlp = torch.optim.Adam(
                    grid.basis_mlp.parameters(),
                    lr=args.lr_basis
                )

In [81]:
dset.shuffle_rays()

 Shuffling rays


RuntimeError: CUDA out of memory. Tried to allocate 844.00 MiB (GPU 0; 7.79 GiB total capacity; 5.60 GiB already allocated; 55.56 MiB free; 6.36 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [60]:
while True:
    dset.shuffle_rays()
    epoch_id += 1
    epoch_size = dset.rays.origins.size(0)
    batches_per_epoch = (epoch_size-1)//args.batch_size+1
    
    if epoch_id % max(factor, args.eval_every) == 0: #and (epoch_id > 0 or not args.tune_mode):
        # NOTE: we do an eval sanity check, if not in tune_mode
        eval_step()
        gc.collect()

    train_step()
    gc.collect()
    gstep_id_base += batches_per_epoch

    #  ckpt_path = path.join(args.train_dir, f'ckpt_{epoch_id:05d}.npz')
    # Overwrite prev checkpoints since they are very huge
    if args.save_every > 0 and (epoch_id + 1) % max(
            factor, args.save_every) == 0 and not args.tune_mode:
        print('Saving', ckpt_path)
        grid.save(ckpt_path)

    if (gstep_id_base - last_upsamp_step) >= args.upsamp_every:
        last_upsamp_step = gstep_id_base
        if reso_id < len(reso_list) - 1:
            print('* Upsampling from', reso_list[reso_id], 'to', reso_list[reso_id + 1])
            if args.tv_early_only > 0:
                print('turning off TV regularization')
                args.lambda_tv = 0.0
                args.lambda_tv_sh = 0.0
            elif args.tv_decay != 1.0:
                args.lambda_tv *= args.tv_decay
                args.lambda_tv_sh *= args.tv_decay

            reso_id += 1
            use_sparsify = True
            z_reso = reso_list[reso_id] if isinstance(reso_list[reso_id], int) else reso_list[reso_id][2]
            grid.resample(reso=reso_list[reso_id],
                    sigma_thresh=args.density_thresh,
                    weight_thresh=args.weight_thresh / z_reso if use_sparsify else 0.0,
                    dilate=2, #use_sparsify,
                    cameras=resample_cameras if args.thresh_type == 'weight' else None,
                    max_elements=args.max_grid_elements)

            if grid.use_background and reso_id <= 1:
                grid.sparsify_background(args.background_density_thresh)

            if args.upsample_density_add:
                grid.density_data.data[:] += args.upsample_density_add

        if factor > 1 and reso_id < len(reso_list) - 1:
            print('* Using higher resolution images due to large grid; new factor', factor)
            factor //= 2
            dset.gen_rays(factor=factor)
            dset.shuffle_rays()

    if gstep_id_base >= args.n_iters:
        print('* Final eval and save')
        eval_step()
        """
        global_stop_time = datetime.now()
        secs = (global_stop_time - global_start_time).total_seconds()
        timings_file = open(os.path.join(args.train_dir, 'time_mins.txt'), 'a')
        timings_file.write(f"{secs / 60}\n")
        """
        if not args.tune_nosave:
            grid.save(ckpt_path)
        break

 Shuffling rays


KeyboardInterrupt: 

# Рендерим результат

In [20]:
import imageio
import numpy as np

In [38]:
render_dir = PATH + "/render"

In [22]:
if False:
  render_dir += "_test"
  dset = dset_test

In [42]:
with torch.no_grad():
    n_images = dset.n_images
    img_eval_interval = max(n_images // 100000, 1)
    avg_psnr = 0.0
    avg_ssim = 0.0
    avg_lpips = 0.0
    n_images_gen = 0
    c2ws = dset.c2w.to(device=device)

    frames = []
    for img_id in tqdm(range(0, n_images, img_eval_interval)):
        dset_h, dset_w = dset.get_image_size(img_id)
        im_size = dset_h * dset_w
        w = c if args.crop == 1.0 else int(dset_w * args.crop)
        h = dset_h if args.crop == 1.0 else int(dset_h * args.crop)

        cam = svox2.Camera(c2ws[img_id],
                           dset.intrins.get('fx', img_id),
                           dset.intrins.get('fy', img_id),
                           dset.intrins.get('cx', img_id) + (w - dset_w) * 0.5,
                           dset.intrins.get('cy', img_id) + (h - dset_h) * 0.5,
                           w, h,
                           ndc_coeffs=dset.ndc_coeffs)
        im = grid.volume_render_image(cam, use_kernel=True, return_raylen=args.ray_len)
       
        if args.ray_len:
            minv, meanv, maxv = im.min().item(), im.mean().item(), im.max().item()
            im = viridis_cmap(im.cpu().numpy())
            cv2.putText(im, f"{'%.4f' % minv} {'%.4f' % meanv} {'%.4f' % maxv}", (10, 20),
                        0, 0.5, [255, 0, 0])
            im = torch.from_numpy(im).to(device=device)
        im.clamp_(0.0, 1.0)

        im_gt = dset.gt[img_id].to(device=device)
        mse = (im - im_gt) ** 2 
        mse_num : float = mse.mean().item()
        psnr = -10.0 * math.log10(mse_num)
        avg_psnr += psnr

        img_path = path.join(render_dir, f'{"%.04d" % img_id}.png');
        im = im.cpu().numpy()

        im_gt = dset.gt[img_id].numpy()
        im = np.concatenate([im_gt, im], axis=1)

        im = (im * 255).astype(np.uint8)
        if not args.no_imsave:
            imageio.imwrite(img_path,im)
        if not args.no_vid:
            frames.append(im)

        im = None
        n_images_gen += 1

    if not args.no_vid and len(frames):
        vid_path = render_dir + '/compilation.mp4'
        imageio.mimwrite(vid_path, frames, fps=30, macro_block_size=8)  # pip install imageio-ffmpeg

  0%|          | 0/95 [00:00<?, ?it/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=8, resizing from (2030, 764) to (2032, 768) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x5b0e740] Warning: data is not aligned! This can lead to a speed loss


In [55]:
obs_dir = PATH + "/obs"

In [59]:
cam = svox2.Camera(c2ws[img_id],
                           dset.intrins.get('fx', ),
                           dset.intrins.get('fy', img_id),
                           dset.intrins.get('cx', img_id) + (w - dset_w) * 0.5,
                           dset.intrins.get('cy', img_id) + (h - dset_h) * 0.5,
                           w, h,
                           ndc_coeffs=dset.ndc_coeffs)
with torch.no_grad():
    for i in tqdm(range(10)):
        cam.fx += 10
        im = grid.volume_render_image(cam, use_kernel=True, return_raylen=args.ray_len)

        im = im.cpu().numpy()
        
        im = (im * 255).astype(np.uint8)

        img_path = path.join(obs_dir, f'{"%.04d" % i}.png')
        imageio.imwrite(img_path,im)

  0%|          | 0/10 [00:00<?, ?it/s]